#Libraries and modules

In [ ]:
import sys
'geopandas' in sys.modules
!pip install geopandas
!pip install pydeck
!pip install Streamlit
import urllib.request
import pandas as pd
import pydeck as pdk
import geopandas as gpd
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.pyplot as plt
import streamlit as st
import numpy as np

#Food waste production

In [6]:
# Load the data from the CSV file into a pandas dataframe
data = pd.read_csv('/content/drive/MyDrive/Thesis/Python codes/Waste Production per Region/Food Waste per ATECO.csv')

# Load the shapefile for Italy regions as a GeoDataFrame and convert its CRS to lat/lon
Italy = gpd.read_file('/content/drive/MyDrive/Thesis/Python codes/Graphs/italian-maps-shapefiles-master/italian-maps-shapefiles-master/italy-with-regions/reg2011_g.shp')
new_regions = pd.read_csv('/content/drive/MyDrive/Thesis/Python codes/Graphs/20200428_Covid-19_Italy_Map/reg2011_g.csv')
Italy['NOME_REG'] = new_regions['NOME_REG']
Italy['NOME_REG'] = new_regions['NOME_REG']
Italy['NOME_REG'] = [s.replace("-","") for s in Italy['NOME_REG']]
Italy['NOME_REG'] = [s.replace(" ","") for s in Italy['NOME_REG']]
Italy['NOME_REG'] = [s.replace("Aosta","ValledAosta") for s in Italy['NOME_REG']]
Italy['NOME_REG'] = [s.replace("Trentino","TrentinoAltoAdige") for s in Italy['NOME_REG']]
# Italy.head()
shapefile = Italy
gdf = Italy
gdf = gdf.to_crs(epsg=4326)

view_state = pdk.ViewState(latitude=41.87194, longitude=12.56738, zoom=4.8, bearing=0, pitch=35)

#gdf

region_layers = []

# Loop through each region in the GeoDataFrame and create a new layer for it
for region_name in gdf['NOME_REG']:
    region_layer = pdk.Layer(
        'GeoJsonLayer',
        gdf.loc[gdf['NOME_REG'] == region_name],
        opacity=0.5,
        stroked=True,
        filled=True,
        get_fill_color = '[255, 255, 255]',
        get_line_color='[0, 0, 0]',
        line_width_min_pixels=1.5,
        pickable=False,
        auto_highlight=True
    )
    region_layers.append(region_layer)

centroids = gpd.GeoDataFrame()
centroids["geometry"] = Italy.geometry.centroid
centroids["NOME_REG"] = Italy.NOME_REG

merged_data = pd.merge(centroids, data, on='NOME_REG')
merged_data = merged_data.to_crs(epsg=4326)
merged_data["lon"] = merged_data.geometry.x
merged_data["lat"] = merged_data.geometry.y
merged_data["values"] = merged_data[merged_data.columns[3]]
merged_data['values_with_separators'] = merged_data['values'].apply(lambda x: '{:,}'.format(x))


merged_data['NOME_REG'] = [s.replace("ValledAosta","Valle d'Aosta") for s in merged_data['NOME_REG']]
merged_data['NOME_REG'] = [s.replace("TrentinoAltoAdige","Trentino Alto Adige") for s in merged_data['NOME_REG']]
merged_data['NOME_REG'] = [s.replace("FriuliVeneziaGiulia","Friuli Venezia Giulia") for s in merged_data['NOME_REG']]
merged_data['NOME_REG'] = [s.replace("EmiliaRomagna","Emilia Romagna") for s in merged_data['NOME_REG']]

# create mapping from food waste to color
plasma = cm = plt.get_cmap('plasma') 
cNorm  = colors.Normalize(vmin=merged_data["values"].min(), vmax=merged_data["values"].max())
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=plasma)
merged_data["color"] = merged_data.apply (lambda row: scalarMap.to_rgba(row["values"]), axis=1)

# merged_data

#Create the bar chart using pydeck
column_layer = pdk.Layer(
    "ColumnLayer",
    data=merged_data,
    get_position=["lon", "lat"],
    get_elevation='values',
    elevation_scale=0.9,
    radius=17000,
    pickable=True,
    get_fill_color="[color[0] * 255, color[1] * 255, color[2] * 255, color[3] * 255]",
    auto_highlight=True
)

# Create the tooltip
h = ""
h += "<div style='text-shadow: 1.7px 0.7px 3.5px #DCDCDC; color: #FFFFFF; font-size: 30px; font-family: Lucida Sans; font-weight: bold;'>{NOME_REG}</div>"
h += "<div style='line-height: 1.5;'><br></div>"
h += "<div style='line-height: 1.5; color: #FFFFFF; font-size: 19px; font-family: Lucida Sans;'>Food and bevarage (NACE 2007: 10-11) waste production:</div>"
h += "<div style='line-height: 1.5; color: #FFFFFF; font-size: 25px; font-family: Lucida Sans;'>{values_with_separators} [t/y]</div>"

tooltip = {
    "html": h,
    "style": {"background": "green", "color": "white", "font-family": '"Helvetica Neue", Arial', "z-index": "10000", "border-radius": "10px"},
}


# Create the map with the features defined in the province_layers, with an initial view selected the Emilia Romagna region and the tooltip
r = pdk.Deck(layers=region_layers, initial_view_state=view_state, tooltip=tooltip)
# Add the bar chart layer to the map and display it
r.layers.append(column_layer)

# Save the map as an html file in the desired folder
r.to_html("/content/drive/MyDrive/Thesis/Graphs/Waste production per region/3D maps/ATECO 10,11.html", notebook_display=True)

# Show the map using Streamlit library (not usefull)
st.pydeck_chart(r)

Output hidden; open in https://colab.research.google.com to view.

In [6]:
# Load the data from the CSV file into a pandas dataframe
data = pd.read_csv('/content/drive/MyDrive/Thesis/Python codes/Waste Production per Region/Food Waste per EER.csv')

# Load the shapefile for Italy regions as a GeoDataFrame and convert its CRS to lat/lon
Italy = gpd.read_file('/content/drive/MyDrive/Thesis/Python codes/Graphs/italian-maps-shapefiles-master/italian-maps-shapefiles-master/italy-with-regions/reg2011_g.shp')
new_regions = pd.read_csv('/content/drive/MyDrive/Thesis/Python codes/Graphs/20200428_Covid-19_Italy_Map/reg2011_g.csv')
Italy['NOME_REG'] = new_regions['NOME_REG']
Italy['NOME_REG'] = new_regions['NOME_REG']
Italy['NOME_REG'] = [s.replace("-","") for s in Italy['NOME_REG']]
Italy['NOME_REG'] = [s.replace(" ","") for s in Italy['NOME_REG']]
Italy['NOME_REG'] = [s.replace("Aosta","ValledAosta") for s in Italy['NOME_REG']]
Italy['NOME_REG'] = [s.replace("Trentino","TrentinoAltoAdige") for s in Italy['NOME_REG']]
# Italy.head()
shapefile = Italy
gdf = Italy
gdf = gdf.to_crs(epsg=4326)

view_state = pdk.ViewState(latitude=41.87194, longitude=12.56738, zoom=4.8, bearing=0, pitch=35)

#gdf

region_layers = []

# Loop through each region in the GeoDataFrame and create a new layer for it
for region_name in gdf['NOME_REG']:
    region_layer = pdk.Layer(
        'GeoJsonLayer',
        gdf.loc[gdf['NOME_REG'] == region_name],
        opacity=0.5,
        stroked=True,
        filled=True,
        get_fill_color = '[255, 255, 255]',
        get_line_color='[0, 0, 0]',
        line_width_min_pixels=1.5,
        pickable=False,
        auto_highlight=True
    )
    region_layers.append(region_layer)

centroids = gpd.GeoDataFrame()
centroids["geometry"] = Italy.geometry.centroid
centroids["NOME_REG"] = Italy.NOME_REG

merged_data = pd.merge(centroids, data, on='NOME_REG')
merged_data = merged_data.to_crs(epsg=4326)
merged_data["lon"] = merged_data.geometry.x
merged_data["lat"] = merged_data.geometry.y
merged_data["values"] = merged_data[merged_data.columns[3]]
merged_data['values_with_separators'] = merged_data['values'].apply(lambda x: '{:,}'.format(x))


merged_data['NOME_REG'] = [s.replace("ValledAosta","Valle d'Aosta") for s in merged_data['NOME_REG']]
merged_data['NOME_REG'] = [s.replace("TrentinoAltoAdige","Trentino Alto Adige") for s in merged_data['NOME_REG']]
merged_data['NOME_REG'] = [s.replace("FriuliVeneziaGiulia","Friuli Venezia Giulia") for s in merged_data['NOME_REG']]
merged_data['NOME_REG'] = [s.replace("EmiliaRomagna","Emilia Romagna") for s in merged_data['NOME_REG']]

# create mapping from food waste to color
plasma = cm = plt.get_cmap('plasma') 
cNorm  = colors.Normalize(vmin=merged_data["values"].min(), vmax=merged_data["values"].max())
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=plasma)
merged_data["color"] = merged_data.apply (lambda row: scalarMap.to_rgba(row["values"]), axis=1)

# merged_data

#Create the bar chart using pydeck
column_layer = pdk.Layer(
    "ColumnLayer",
    data=merged_data,
    get_position=["lon", "lat"],
    get_elevation='values',
    elevation_scale=0.9,
    radius=17000,
    pickable=True,
    get_fill_color="[color[0] * 255, color[1] * 255, color[2] * 255, color[3] * 255]",
    auto_highlight=True
)

# Create the tooltip
h = ""
h += "<div style='text-shadow: 1.7px 0.7px 3.5px #DCDCDC; color: #FFFFFF; font-size: 30px; font-family: Lucida Sans; font-weight: bold;'>{NOME_REG}</div>"
h += "<div style='line-height: 1.5;'><br></div>"
h += "<div style='line-height: 1.5; color: #FFFFFF; font-size: 19px; font-family: Lucida Sans;'>EER code 02 waste production:</div>"
h += "<div style='line-height: 1.5; color: #FFFFFF; font-size: 25px; font-family: Lucida Sans;'>{values_with_separators} [t/y]</div>"

tooltip = {
    "html": h,
    "style": {"background": "green", "color": "white", "font-family": '"Helvetica Neue", Arial', "z-index": "10000", "border-radius": "10px"},
}


# Create the map with the features defined in the province_layers, with an initial view selected the Emilia Romagna region and the tooltip
r = pdk.Deck(layers=region_layers, initial_view_state=view_state, tooltip=tooltip)
# Add the bar chart layer to the map and display it
r.layers.append(column_layer)

# Save the map as an html file in the desired folder
r.to_html("/content/drive/MyDrive/Thesis/Graphs/Waste production per region/3D maps/EER 02.html", notebook_display=True)
# r.to_json("/content/drive/MyDrive/Thesis/Graphs/Waste production per region/3D maps/EER 02.json")

# Show the map using Streamlit library (not usefull)
st.pydeck_chart(r)

Output hidden; open in https://colab.research.google.com to view.